### RAG

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50,
    separators=["\n", "\n\n",]
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)
print(docs[0])
len(docs)

page_content='Part 1, Chapter 1\n\nPart One\n\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.' metadata={'source': './files/chapter_one.txt'}


45

In [39]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600, 
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)
print(docs[0])
len(docs)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.' metadata={'source': './files/chapter_one.txt'}


45

In [40]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600, 
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)
print(docs[0])
len(docs)

page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer a

16

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [46]:
result = vectorstore.similarity_search("where does winston live?")
print(result[0])

page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liquid with a 

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

print(chain.run("Describe Victory Mansions"))

Victory Mansions is a place described in the book "1984" by George Orwell. It is a building that is not well-maintained, with a hallway smelling of boiled cabbage and old rag mats. The building has a dysfunctional lift and the electricity is cut off during daylight hours as part of an economy drive. The main character, Winston Smith, lives in a flat in Victory Mansions on the seventh floor. The building has a poster of an enormous face with the caption "BIG BROTHER IS WATCHING YOU" on each landing. The atmosphere in Victory Mansions reflects the oppressive and rundown nature of the society depicted in the novel.


In [7]:
print(chain.run("Where does Winston live?"))

Winston lives in Victory Mansions.


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

print(chain.run("Describe Victory Mansions"))

Victory Mansions is a building where Winston Smith lives. It is described as having glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and it is seven flights up. The building is poorly maintained, with a broken lift, and often has the electric current cut off during daylight hours. Inside, there is a telescreen that cannot be completely shut off and is always watching. Winston's flat is in Victory Mansions, and it is where he resides in the story.


In [10]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

print(chain.run("Describe Victory Mansions"))

The additional context provided gives insight into Winston's living situation within Victory Mansions and sheds light on his rebellious actions against the Party. Winston strategically positions himself in an alcove to avoid being seen by the telescreen, allowing him a rare moment of privacy to engage in forbidden activities. This includes reading a forbidden book, an act of defiance against the Party's strict control over information and literature.

The description of the book as a "peculiarly beautiful book" with aged paper hints at its significance and rarity in the dystopian society of Oceania. Winston's decision to purchase the book from a prohibited shop, despite the risks involved, demonstrates his longing for knowledge and individuality in a society that suppresses such desires.

This additional context further emphasizes the oppressive and restrictive nature of Victory Mansions, highlighting the constant surveillance and control exerted by the Party over its inhabitants. Wins

##### SYSTEM

Context information is below.

The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.
Winston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liquid with a plain white label marked VICTORY GIN. It gave off a sickly, oily smell, as of Chinese ricespirit. Winston poured out nearly a teacupful, nerved himself for a shock, and gulped it down like a dose of medicine.
Instantly his face turned scarlet and the water ran out of his eyes. The stuff was like nitric acid, and moreover, in swallowing it one had the sensation of being hit on the back of the head with a rubber club. The next moment, however, the burning in his belly died down and the world began to look more cheerful. He took a cigarette from a crumpled packet marked VICTORY CIGARETTES and incautiously held it upright, whereupon the tobacco fell out on to the floor. With the next he was more successful. He went back to the living-room and sat down at a small table that stood to the left of the telescreen. From the table drawer he took out a penholder, a bottle of ink, and a thick, quarto-sized blank book with a red back and a marbled cover.

Given the context information and not prior knowledge, answer any questions

##### HUMAN

Describe Victory Mansions

##### AI

Victory Mansions is a building complex where Winston Smith, the protagonist of George Orwell's novel "1984," resides. It is a run-down, dilapidated apartment building located in Airstrip One (formerly known as Great Britain) under the rule of the totalitarian regime led by Big Brother. The living conditions in Victory Mansions are poor, with shortages of basic necessities like food and utilities. The building is described as having broken elevators, shabby staircases, and cramped, dingy apartments. Residents like Winston endure constant surveillance and propaganda from the Party, adding to the oppressive atmosphere of the place.

##### HUMAN

We have the opportunity to refine the existing answer (only if needed) with some more context below.

Part 1, Chapter 1
Part One
1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.
The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer above his right ankle, went slowly, resting several times on the way. On each landing, opposite the lift-shaft, the poster with the enormous face gazed from the wall. It was one of those pictures which are so contrived that the eyes follow you about when you move. BIG BROTHER IS WATCHING YOU, the caption beneath it ran.
Inside the flat a fruity voice was reading out a list of figures which had something to do with the production of pig-iron. The voice came from an oblong metal plaque like a dulled mirror which formed part of the surface of the right-hand wall. Winston turned a switch and the voice sank somewhat, though the words were still distinguishable. The instrument (the telescreen, it was called) could be dimmed, but there was no way of shutting it off completely. He moved over to the window: a smallish, frail figure, the meagreness of his body merely emphasized by the blue overalls which were the uniform of the party. His hair was very fair, his face naturally sanguine, his skin roughened by coarse soap and blunt razor blades and the cold of the winter that had just ended.

Given the new context, refine the original answer to better answer the question. If the context isn't useful, return the original answer.

##### AI
Victory Mansions is a run-down building complex where Winston Smith, the protagonist of George Orwell's novel "1984," resides. Located in Airstrip One under the rule of the totalitarian regime led by Big Brother, Victory Mansions is described as having a dilapidated appearance with broken elevators, shabby staircases, and cramped, dingy apartments. The living conditions are poor, with shortages of basic necessities like food and utilities. Residents endure constant surveillance and propaganda from the Party, which adds to the oppressive atmosphere of the place. The building is adorned with a larger-than-life poster of Big Brother, with the caption "BIG BROTHER IS WATCHING YOU," serving as a constant reminder of the government's control and surveillance over its citizens. The residents, like Winston, also have to contend with the presence of telescreens that cannot be shut off completely, further intruding on their privacy and freedom.

...

### Stuff LCEL Chain

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
# from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "extra": RunnableParallel(),
    }
    | prompt
    | llm
)

print(chain.invoke("Describe Victory Mansions"))

content='Victory Mansions is a building where Winston Smith lives. It is described as having glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU." The building has a faulty lift, forcing residents to climb stairs. The flat where Winston lives is seven flights up, and the building is part of the city of London in Airstrip One, a province of Oceania.'


##### SYSTEM


You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liquid with a plain white label marked VICTORY GIN. It gave off a sickly, oily smell, as of Chinese ricespirit. Winston poured out nearly a teacupful, nerved himself for a shock, and gulped it down like a dose of medicine.\nInstantly his face turned scarlet and the water ran out of his eyes. The stuff was like nitric acid, and moreover, in swallowing it one had the sensation of being hit on the back of the head with a rubber club. The next moment, however, the burning in his belly died down and the world began to look more cheerful. He took a cigarette from a crumpled packet marked VICTORY CIGARETTES and incautiously held it upright, whereupon the tobacco fell out on to the floor. With the next he was more successful. He went back to the living-room and sat down at a small table that stood to the left of the telescreen. From the table drawer he took out a penholder, a bottle of ink, and a thick, quarto-sized blank book with a red back and a marbled cover.", metadata={'source': './files/chapter_one.txt'}), Document(page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer above his right ankle, went slowly, resting several times on the way. On each landing, opposite the lift-shaft, the poster with the enormous face gazed from the wall. It was one of those pictures which are so contrived that the eyes follow you about when you move. BIG BROTHER IS WATCHING YOU, the caption beneath it ran.\nInside the flat a fruity voice was reading out a list of figures which had something to do with the production of pig-iron. The voice came from an oblong metal plaque like a dulled mirror which formed part of the surface of the right-hand wall. Winston turned a switch and the voice sank somewhat, though the words were still distinguishable. The instrument (the telescreen, it was called) could be dimmed, but there was no way of shutting it off completely. He moved over to the window: a smallish, frail figure, the meagreness of his body merely emphasized by the blue overalls which were the uniform of the party. His hair was very fair, his face naturally sanguine, his skin roughened by coarse soap and blunt razor blades and the cold of the winter that had just ended.', metadata={'source': './files/chapter_one.txt'}), Document(page_content='Winston kept his back turned to the telescreen. It was safer, though, as he well knew, even a back can be revealing. A kilometre away the Ministry of Truth, his place of work, towered vast and white above the grimy landscape. This, he thought with a sort of vague distaste -- this was London, chief city of Airstrip One, itself the third most populous of the provinces of Oceania. He tried to squeeze out some childhood memory that should tell him whether London had always been quite like this. Were there always these vistas of rotting nineteenth-century houses, their sides shored up with baulks of timber, their windows patched with cardboard and their roofs with corrugated iron, their crazy garden walls sagging in all directions? And the bombed sites where the plaster dust swirled in the air and the willow-herb straggled over the heaps of rubble; and the places where the bombs had cleared a larger patch and there had sprung up sordid colonies of wooden dwellings like chicken-houses? But it was no use, he could not remember: nothing remained of his childhood except a series of bright-lit tableaux occurring against no background and mostly unintelligible.\nThe Ministry of Truth -- Minitrue, in Newspeak -- was startlingly different from any other object in sight. It was an enormous pyramidal structure of glittering white concrete, soaring up, terrace after terrace, 300 metres into the air. From where Winston stood it was just possible to read, picked out on its white face in elegant lettering, the three slogans of the Party:\nWAR IS PEACE\nFREEDOM IS SLAVERY\nIGNORANCE IS STRENGTH\nThe Ministry of Truth contained, it was said, three thousand rooms above ground level, and corresponding ramifications below. Scattered about London there were just three other buildings of similar appearance and size. So completely did they dwarf the surrounding architecture that from the roof of Victory Mansions you could see all four of them simultaneously. They were the homes of the four Ministries between which the entire apparatus of government was divided. The Ministry of Truth, which concerned itself with news, entertainment, education, and the fine arts. The Ministry of Peace, which concerned itself with war. The Ministry of Love, which maintained law and order. And the Ministry of Plenty, which was responsible for economic affairs. Their names, in Newspeak: Minitrue, Minipax, Miniluv, and Miniplenty.', metadata={'source': './files/chapter_one.txt'}), Document(page_content="For some reason the telescreen in the living-room was in an unusual position. Instead of being placed, as was normal, in the end wall, where it could command the whole room, it was in the longer wall, opposite the window. To one side of it there was a shallow alcove in which Winston was now sitting, and which, when the flats were built, had probably been intended to hold bookshelves. By sitting in the alcove, and keeping well back, Winston was able to remain outside the range of the telescreen, so far as sight went. He could be heard, of course, but so long as he stayed in his present position he could not be seen. It was partly the unusual geography of the room that had suggested to him the thing that he was now about to do.\nBut it had also been suggested by the book that he had just taken out of the drawer. It was a peculiarly beautiful book. Its smooth creamy paper, a little yellowed by age, was of a kind that had not been manufactured for at least forty years past. He could guess, however, that the book was much older than that. He had seen it lying in the window of a frowsy little junk-shop in a slummy quarter of the town (just what quarter he did not now remember) and had been stricken immediately by an overwhelming desire to possess it. Party members were supposed not to go into ordinary shops ('dealing on the free market', it was called), but the rule was not strictly kept, because there were various things, such as shoelaces and razor blades, which it was impossible to get hold of in any other way. He had given a quick glance up and down the street and then had slipped inside and bought the book for two dollars fifty. At the time he was not conscious of wanting it for any particular purpose. He had carried it guiltily home in his briefcase. Even with nothing written in it, it was a compromising possession.", metadata={'source': './files/chapter_one.txt'})]

##### HUMAN
Describe Victory Mansions

### Map Reduce LCEL Chain

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")

AIMessage(content='Three ministries are mentioned in the text: the Ministry of Love, the Ministry of Plenty, and the Ministry of Truth.')

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini" 
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

# Initialize embeddings
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()



In [24]:
print(docs[0])

page_content='Chapter 3\n\'There are three stages in your reintegration,\' said O\'Brien. \'There is learning, there is understanding, and there is acceptance. It is time for you to enter upon the second stage.\'\nAs always, Winston was lying flat on his back. But of late his bonds were looser. They still held him to the bed, but he could move his knees a little and could turn his head from side to side and raise his arms from the elbow. The dial, also, had grown to be less of a terror. He could evade its pangs if he was quick-witted enough: it was chiefly when he showed stupidity that O\'Brien pulled the lever. Sometimes they got through a whole session without use of the dial. He could not remember how many sessions there had been. The whole process seemed to stretch out over a long, indefinite time--weeks, possibly--and the intervals between the sessions might sometimes have been days, sometimes only an hour or two.\n\'As you lie there,\' said O\'Brien, \'you have often wondered--yo

In [55]:
memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

def retrieve_context(question):
    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    return context

# Define the chat prompt template with placeholders for context, history, and question
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up"
        ),
        ("system", "Context:\n{context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# Define a lambda function to load conversation history
def load_memory(inputs):
    return memory.load_memory_variables({})["history"]

# Define a lambda function to retrieve context based on the question
def get_context(inputs):
    inputs["context"] = retrieve_context(inputs["question"])
    print(inputs)
    return inputs

chain = (
    RunnablePassthrough()
    .assign(history=load_memory)  
    | RunnableLambda(get_context) 
    | final_prompt  
    | llm  
)

def invoke_chain(question: str):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [56]:
invoke_chain("Is Aaronson guilty?")

{'question': 'Is Aaronson guilty?', 'history': [], 'context': "One day--but 'one day' was not the right expression; just as probably it was in the middle of the night: once--he fell into a strange, blissful reverie. He was walking down the corridor, waiting for the bullet. He knew that it was coming in another moment. Everything was settled, smoothed out, reconciled. There were no more doubts, no more arguments, no more pain, no more fear. His body was healthy and strong. He walked easily, with a joy of movement and with a feeling of walking in sunlight. He was not any longer in the narrow white corridors in the Ministry of Love, he was in the enormous sunlit passage, a kilometre wide, down which he had seemed to walk in the delirium induced by drugs. He was in the Golden Country, following the foot-track across the old rabbit-cropped pasture. He could feel the short springy turf under his feet and the gentle sunshine on his face. At the edge of the field were the elm trees, faintly st

In [57]:
invoke_chain("What message did he write in the table?")

{'question': 'What message did he write in the table?', 'history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content="According to Winston, Aaronson is guilty of the crimes he was charged with, as he has accepted the Party's narrative that he had never seen the photograph that disproved their guilt and that it had never existed.")], 'context': "Only when he chanced to put his hand on his bald scalp did he remember the seamed, ruined face that had looked back at him out of the mirror.\nHis mind grew more active. He sat down on the plank bed, his back against the wall and the slate on his knees, and set to work deliberately at the task of re-educating himself.\nHe had capitulated, that was agreed. In reality, as he saw now, he had been ready to capitulate long before he had taken the decision. From the moment when he was inside the Ministry of Love--and yes, even during those minutes when he and Julia had stood helpless while the iron voice from the telescreen told them wha

In [58]:
invoke_chain("Who is Julia?")

{'question': 'Who is Julia?', 'history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content="According to Winston, Aaronson is guilty of the crimes he was charged with, as he has accepted the Party's narrative that he had never seen the photograph that disproved their guilt and that it had never existed."), HumanMessage(content='What message did he write in the table?'), AIMessage(content='Winston traced "2+2=5" in the dust on the table.')], 'context': "One day--but 'one day' was not the right expression; just as probably it was in the middle of the night: once--he fell into a strange, blissful reverie. He was walking down the corridor, waiting for the bullet. He knew that it was coming in another moment. Everything was settled, smoothed out, reconciled. There were no more doubts, no more arguments, no more pain, no more fear. His body was healthy and strong. He walked easily, with a joy of movement and with a feeling of walking in sunlight. He was not any longer in the na

In [68]:
memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

def history_retrieve_context(inputs):
    question = inputs.get("question", "")
    history = inputs.get("history", "")
    combined_query = f"Conversation History:\n{history}\n\nNew Question:\n{question}"
    relevant_docs = retriever.get_relevant_documents(combined_query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    return {"context": context}

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up"
        ),
        ("system", "Context:\n{context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(inputs):
    return memory.load_memory_variables({})["history"]

def get_context(inputs):
    inputs["context"] = history_retrieve_context(inputs)
    print(inputs)
    return inputs

chain = (
    RunnablePassthrough()
    .assign(history=load_memory)  
    | RunnableLambda(get_context) 
    | final_prompt  
    | llm  
)

def invoke_chain(question: str):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [69]:
invoke_chain("Is Aaronson guilty?")

{'question': 'Is Aaronson guilty?', 'history': [], 'context': {'context': "Then almost without a pause he wrote beneath it:\nTWO AND TWO MAKE FIVE\nBut then there came a sort of check. His mind, as though shying away from something, seemed unable to concentrate. He knew that he knew what came next, but for the moment he could not recall it. When he did recall it, it was only by consciously reasoning out what it must be: it did not come of its own accord. He wrote:\nGOD IS POWER\nHe accepted everything. The past was alterable. The past never had been altered. Oceania was at war with Eastasia. Oceania had always been at war with Eastasia. Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with. He had never seen the photograph that disproved their guilt. It had never existed, he had invented it. He remembered remembering contrary things, but those were false memories, products of self-deception. How easy it all was! Only surrender, and everything else followed. I

In [70]:
invoke_chain("What message did he write in the table?")

{'question': 'What message did he write in the table?', 'history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='According to the context, Aaronson is guilty of the crimes he was charged with, as Winston states, "Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with." However, Winston also acknowledges that he had never seen the photograph that disproved their guilt and that it had never existed, implying that their guilt is a fabrication of the Party.')], 'context': {'context': 'Then almost without a pause he wrote beneath it:\nTWO AND TWO MAKE FIVE\nBut then there came a sort of check. His mind, as though shying away from something, seemed unable to concentrate. He knew that he knew what came next, but for the moment he could not recall it. When he did recall it, it was only by consciously reasoning out what it must be: it did not come of its own accord. He wrote:\nGOD IS POWER\nHe accepted everything. The past was alterable. The past nev

In [62]:
invoke_chain("Who is Julia?")

{'question': 'Who is Julia?', 'history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='According to Winston, Aaronson is guilty of the crimes he was charged with, but he also reflects that he had never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it suggests that Aaronson is not actually guilty, but the Party has declared him so.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='Winston traced "2+2=5" in the dust on the table.')], 'context': "One day--but 'one day' was not the right expression; just as probably it was in the middle of the night: once--he fell into a strange, blissful reverie. He was walking down the corridor, waiting for the bullet. He knew that it was coming in another moment. Everything was settled, smoothed out, reconciled. There were no more doubts, no more arguments, no more pain, no more fear. His body was healthy and strong. He walked easily, w